# Генератор тестовых данных

Пример генератора тестовых данных на остове функции sin. В примере создаются два набора данных с разными аплитудами. Амплитуда на участке для обучения меньше чем на участке для проверки модели. Также на основе данных строится временной ряд.

## Генерация данных

In [ ]:
import numpy
import random

#№Date and time (GMT): Sunday, 1 January 2023 г., 0:00:00
#Date and time (your time zone): воскресенье, 1 января 2023 г., 3:00:00 GMT+03:00
start  = 1672531200

#точки по оси X с периодичностью 1 час (3600 секунд)
period = 3600 
# количество точек по оси X для одного отрезка
period_count = 20
#амплитуда периодичной функции
amplitude_train = 10
amplitude_predict = amplitude_train * 1.5
#сдвиг по оси Y вверх
shift = 15
# усилитель шума
amplifier = 1.5

# отрезок первый, данные для ОБУЧЕНИЯ
train_x = numpy.arange(start,start+period*period_count,period) 
# генерация шума
noise = numpy.random.normal(0, 1, len(train_x))*amplifier
# добавления шума к функции sin
train_y = (amplitude_train*numpy.sin(train_x/period) + shift) +  noise  

# отрезок второй, данные для ПРЕДСКАЗАНИЯ
predict_x = numpy.arange(start+period*period_count, # начало после отрезка X1
                           (start+period*period_count)+period*period_count, 
                           period) 
# генерация шума
noise = numpy.random.normal(0, 1, len(predict_x))*amplifier
# добавления шума к произвольной функции
predict_y = (amplitude_predict*numpy.sin(predict_x/period) + shift) +  noise  

In [ ]:
# график двух отрезков, разными цветами
import matplotlib.pyplot as plt
fig, axs = plt.subplots(1)
axs.plot(train_x,train_y, 'o', color="black", markersize=1)
axs.plot(predict_x,predict_y, 'o', color="red", markersize=1)
fig.set_figwidth(12)
plt.show()
plt.close("all")

## Формирование parquet

In [ ]:
# обхединение X и Y в матрицу
data = numpy.concatenate(( train_x.reshape(len(train_x),1), train_y.reshape(len(train_y),1) ), axis=1)
data

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, LongType, StringType, IntegerType, DateType, TimestampType, FloatType, BooleanType
from pyspark.sql.functions import col, cast, date_trunc, sum, dayofweek, hour, dayofmonth, lit, sequence

import random

# запуск spark
spark = SparkSession.builder.appName('parquet').getOrCreate()


In [ ]:
# формирование датафрейма
df = spark.createDataFrame(data, ["x", "y"])
df.show()

In [ ]:
# формирование временно'го ряда
tsDF =  df.select(col("x").cast("timestamp").alias("xtime"), col("y") )
tsDF.show()

In [ ]:
localPathParquet = "/tmp/singleparquet"
fileCount = 1 # число файлов
tsDF.coalesce(1).write.mode("overwrite").parquet(localPathParquet)

In [ ]:
spark.stop()

In [ ]:
# полученный набор данных (dataset) можно закинуть на S3
import os
import boto3

# имя файла в s3
filePathParquet = "train.parquet"
# имя корзины в s3
bucketNameParquet = 'data'

s3_target = boto3.resource('s3', 
    endpoint_url=os.environ["AWS_ENDPOINT_URL"],
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False
)

# создание корзины
bucket = s3_target.Bucket(bucketNameParquet)
if bucket.creation_date:
   print("The bucket exists")
else:
   print("The bucket does not exist")
   s3_target.create_bucket(Bucket=bucketNameParquet)

# загрузка файла на s3 
# в локальной папке находится единственный файл и закидывается на S3
print("старт записи в s3")
fileNameParquetLocal = [x for x in os.listdir(localPathParquet) if x.endswith(".parquet")][0]
print('write to s3', 'backet='+bucketNameParquet, 'path='+filePathParquet)
s3_target.Bucket(bucketNameParquet).upload_file(localPathParquet+'/'+fileNameParquetLocal, filePathParquet)

Аналогичным образом делается parquet файл данных для предсказания.